# SQLAlchemy

SQLAlchemy is a python library that can be used to manage how we connect to our SQL server (PostgreSQL in this case).

The advantage of using SQLAlchemy is it allows us to interact with the database using Object-Relational Mapping (ORM).

To install use the following commands:
```sh
pip install SQLAlchemy
```
- If there are issues during installation, consult me or Google because the issues vary for each environment.


More details can be found here:

https://docs.sqlalchemy.org/en/14/intro.html#installation

## Object-Relational Mapping (ORM)

ORM is a way to interact with the database using "objects" (or classes) instead of manually writing the SQL queries.

This allows some of the operations to be simplified, including certain security concerns.

| | SQL | ORM |
| --- | --- | --- |
| Fetching an item by its ID | `cur.execute("SELECT * FROM inventors WHERE key_id='123'")`</br>`inventor_sql = cur.fetchone()`</br>(`List` of values) | `inventor = session.query(Inventor).get('123')`</br>(`Inventor` class) |
| Fetching an item's column | `inventor_sql[0]` | `inventor.key_id` |
| Variable input | `cur.execute(f"SELECT * FROM inventors WHERE key_id='{key_id}'")`</br>`inventor_sql = cur.fetchone()`</br>(`List` of values)</br>**NOTE: Susceptible to SQL injections** | `inventor = session.query(Inventor).get(key_id)`</br>(`Inventor` class)</br>**NOTE: Relatively safe because SQLAlchemy can "clean" the data automatically** |

In general, what an ORM library does is:
- Uses the "Classes" that you define and interpret them as "table" definitions
- When you do operations on the "Class", the library automatically converts it into SQL Statements
  - Usually, libraries will clean some of the inputs to prevent SQL Injection attacks


## Querying the Inventor Table

The "ORM" version of our `Inventor Table` can be written and queried using SQLAlchemy.

In [30]:
from sqlalchemy import Column, VARCHAR, create_engine
from sqlalchemy.orm import Session, declarative_base
from dotenv import load_dotenv
from os import environ

load_dotenv()  # take environment variables from .env.

database_host = environ.get('DB_HOST')
database_name = environ.get('DB_NAME')
database_username = environ.get('DB_USERNAME')
database_password = environ.get('DB_PASSWORD')


# See here for more details about creating the `engine`
# https://docs.sqlalchemy.org/en/14/core/engines.html
engine = create_engine(f'postgresql://{database_username}:{database_password}@{database_host}:5432/{database_name}')


Base = declarative_base()
# See here for more details about declaring `models`
# https://docs.sqlalchemy.org/en/14/orm/quickstart.html#declare-models
class Inventor(Base):
    __tablename__ = 'inventors'

    key_id = Column(VARCHAR, primary_key=True)
    first_name = Column(VARCHAR, nullable=False)
    last_name = Column(VARCHAR, nullable=False)
    location_city = Column(VARCHAR, nullable=False)
    location_state = Column(VARCHAR, nullable=False)
    location_longitude = Column(VARCHAR, nullable=False)
    location_latitude = Column(VARCHAR, nullable=False)

with Session(engine) as session:
    all_inventors = session.query(Inventor).all()
    for inventor in all_inventors:
        print(f'{inventor.key_id}: {inventor.first_name} {inventor.last_name}')


1: test first name test last name
7: Another Inventor
8: Another Inventor
3: Update First Name Inventor
4: Another Update Last Name
2: Upserted Inventor
100: Another Inventor


In [31]:
import psycopg2
from dotenv import load_dotenv
from os import environ

load_dotenv()  # take environment variables from .env.


database_host = environ.get('DB_HOST')
database_name = environ.get('DB_NAME')
database_username = environ.get('DB_USERNAME')
database_password = environ.get('DB_PASSWORD')

conn = None
try:
    conn = psycopg2.connect(
        host=database_host,
        database=database_name,
        user=database_username,
        password=database_password,
    )

    cur = conn.cursor()
    cur.execute('SELECT * FROM inventors')
    all_inventors = cur.fetchall()
    for inventor in all_inventors:
        print(f'{inventor[0]}: {inventor[1]} {inventor[2]}')
finally:
    if conn:
        conn.close()

1: test first name test last name
7: Another Inventor
8: Another Inventor
3: Update First Name Inventor
4: Another Update Last Name
2: Upserted Inventor
100: Another Inventor


## Inserting an Inventor

Read more about `insert`, `update` and `delete` here:
https://docs.sqlalchemy.org/en/14/core/dml.html

In [20]:
from sqlalchemy import insert

with Session(engine) as session:
    # Using `insert` statement
    stmt = insert(Inventor).values(
        key_id='7',
        first_name='Another',
        last_name='Inventor',
        location_city='San Jose',
        location_state='California',
        location_longitude='0',
        location_latitude='0',
    )
    session.execute(stmt)
    session.commit()

    # Using `Inventor` table directly
    inventor = Inventor(
        key_id='8',
        first_name='Another',
        last_name='Inventor',
        location_city='San Jose',
        location_state='California',
        location_longitude='0',
        location_latitude='0',
    )
    session.add(inventor)
    session.commit()

## Updating an Inventor

In [22]:
from sqlalchemy import update


with Session(engine) as session:
    # Using `update` statement
    stmt = update(Inventor).where(
        Inventor.key_id=='3'
    ).values(
        first_name='Update First Name'
    )
    session.execute(stmt)
    session.commit()

    # Using `Inventor` table directly
    inventor = session.query(Inventor).get('4')

    ...

    inventor.last_name = 'Update Last Name'
    session.commit()



## Upserting an Inventor

Implementation is usually SQL Server specific. In PSQL, we use `on_conflict_do_update`.

Read more about PostgreSQL upsert here:
https://docs.sqlalchemy.org/en/13/dialects/postgresql.html#insert-on-conflict-upsert

In [28]:
from sqlalchemy.dialects.postgresql import insert

with Session(engine) as session:
    # Using `insert` statement
    stmt = insert(Inventor).values(
        key_id='100',
        first_name='Another',
        last_name='Inventor',
        location_city='San Jose',
        location_state='California',
        location_longitude='0',
        location_latitude='0',
    ).on_conflict_do_update(
        index_elements=[Inventor.key_id],
        set_={
            'first_name': 'Upserted'
        }
    )
    session.execute(stmt)
    session.commit()

## Joining an Inventor and its Patents